# Equity Premium Prediction Analysis

This notebook follows the (Rapach 2010) [^1] to implement the prediction performance analysis.

[^1]: Rapach, D. E., Strauss, J. K., & Zhou, G. (2010). Out-of-Sample Equity Premium Prediction: Combination Forecasts and Links to the Real Economy. The Review of Financial Studies, 23(2), 821–862. https://doi.org/10.1093/rfs/hhp063


In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import sys
sys.path.append('../module')

from analysis import get_period_return
from analysis import get_oos_r_square,\
                     get_p_value_of_MSPE_adjusted_test,\
                     get_significance_of_MSPE_adjusted_test,\
                     get_utility_gain_from_prediction,\
                     get_significance_of_p_value
from data_handler import get_monthly_date_format
from data_handler import get_quarterly_date_format
from data_handler import get_econ_predictors
from IO_handler import post_dataframe_to_latex_table

%matplotlib inline

In (Rapach 2010), they adopt three measurements to evaluate the performance of the prediction of equity premium.

1. [$R^2_{OS}$: out of sample $R^2$](##out-of-sample-$r^2$)
2. [MSPE - adjusted statistic: The significance of the $R^2_{OS}$](##mspe-adjusted-test-(mean-squared-prediction-error))
3. [$\Delta$: The utility gain](##the-utility-gain)

load prediction results

In [2]:
START_DATE = '1965-01'
END_DATE = '2021-12'
prediction_df = pd.read_csv('../../data/ml_prediction_1947_2005.csv', index_col=0, parse_dates=True, date_parser=get_monthly_date_format)
prediction_df = prediction_df.loc[START_DATE:END_DATE]
equity_premium = prediction_df.pop('Equity Premium')
historical_average = prediction_df.pop('Historical Average')

## out of sample $R^2$

$$
\begin{equation}
R_{O S}^2=1-\frac{\sum_{k=q_0+1}^q\left(r_{m+k}-\hat{r}_{m+k}\right)^2}{\sum_{k=q_0+1}^q\left(r_{m+k}-\bar{r}_{m+k}\right)^2}
\end{equation}
$$

Where $m$ is the size of in sample data and $q_0$ is the origin of the hold out period.

In [3]:
def get_oos_r_square(y_hat: np.ndarray, y: np.ndarray, y_bar: np.ndarray) -> float:
    """
    This function calculates the out-of-sample R square for a prediction.

    Parameters
    ----------
    y_hat : np.ndarray
        Prediction values.
    y : np.ndarray
        True values.
    y_bar : np.ndarray
        Historical average.

    Returns
    -------
    float
        Out-of-sample R square.
    """
    ss_res = np.sum((y - y_hat) ** 2)
    ss_tot = np.sum((y - y_bar) ** 2)
    R_2 = 1 - ss_res / ss_tot
    R_2_percentage = R_2 * 100

    return R_2_percentage

In [8]:
R_2_OOS = prediction_df.apply(lambda x: get_oos_r_square(y=equity_premium, y_hat=x, y_bar=historical_average))
R_2_OOS

Elastic Net       0.078502
Random Forest   -10.215775
dtype: float64

## MSPE adjusted test (Mean Squared Prediction Error)

We test whether a prediction method is significantly different from the historical average. We follow the statistical test by Clark and West (2007). Rapach (2010) also uses this statistical test method.

The null hypothesis is H0: $R_{OS}^2 < 0$

The test statistics is:
\begin{equation}
f_{t+1}=\left(r_{t+1}-\bar{r}_{t+1}\right)^2-\left[\left(r_{t+1}-\hat{r}_{t+1}\right)^2-\left(\bar{r}_{t+1}-\hat{r}_{t+1}\right)^2\right]
\end{equation}

We regress this statistics against a constant and get the one-side p-value.

In [22]:
def get_p_value_of_MSPE_adjusted_test(y:np.ndarray, y_bar:np.ndarray, y_hat:np.ndarray) -> float:
    """
    
    Parameters
    ----------
    y : np.ndarray (n_samples, 1)
    y_bar : np.ndarray (n_samples, 1)
    y_hat : np.ndarray (n_samples, 1)

    Returns
    -------
    p_value_of_MSPE_adjusted : float
    """
    F = (y - y_bar) ** 2 - ((y - y_hat) ** 2 - (y_bar - y_hat) ** 2)
    dummy = np.ones_like(F)
    lm_result = sm.OLS(F, dummy).fit()

    # one-sided test
    beta = lm_result.params[0]
    if beta > 0:
        p_value = lm_result.pvalues.values[0] / 2
    else:
        p_value = 1 - lm_result.pvalues.values[0] / 2

    return p_value


In [13]:
y = equity_premium
y_bar = historical_average
y_hat = prediction_df.iloc[:, 1]

In [14]:
F = (y - y_bar) ** 2 - ((y - y_hat) ** 2 - (y_bar - y_hat) ** 2)
dummy = np.ones_like(F)
lm_result = sm.OLS(F, dummy).fit()
p_value = lm_result.pvalues

In [17]:
lm_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 08 Feb 2023   Prob (F-statistic):                nan
Time:                        17:17:21   Log-Likelihood:                 2402.3
No. Observations:                 484   AIC:                            -4803.
Df Residuals:                     483   BIC:                            -4798.
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.554e-05    7.7e-05      0.332      0.740      -0.000       0.000
==============================================================================
Omnibus:                      315.304   Durbin-Watson:                   1.920
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33155.037
Skew:                          -1.901   Prob(JB):                         0.00
Kurtosis:                      43.368   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
lm_result.params

2.554257314032056e-05

In [25]:
p_value = prediction_df.apply(lambda x: get_p_value_of_MSPE_adjusted_test(y=equity_premium, y_hat=x, y_bar=historical_average))
p_value

Elastic Net      0.103108
Random Forest    0.740081
dtype: float64

In [26]:
def get_significance_of_MSPE_adjusted_test(y:np.ndarray, y_bar:np.ndarray, y_hat:np.ndarray) -> str:
    """
    
    Parameters
    ----------
    y : np.ndarray (n_samples, 1)
    y_bar : np.ndarray (n_samples, 1)
    y_hat : np.ndarray (n_samples, 1)

    Returns
    -------
    significance_of_MSPE_adjusted : str
    """

    p_value = get_p_value_of_MSPE_adjusted_test(y=y, y_hat=y_hat, y_bar=y_bar)
    p_value = round(p_value, ndigits=3)
    if p_value >= 0.1:
        significance = str(p_value) + ' '
    elif p_value > 0.05:
        significance = str(p_value) +' *'
    elif p_value > 0.01:
        significance = str(p_value) +' **'
    elif p_value <= 0.01:
        significance = str(p_value) +' ***'

    return significance

In [26]:
significance = prediction_df.apply(lambda x: get_significance_of_MSPE_adjusted_test(y=equity_premium, y_hat=x, y_bar=historical_average))
significance

Elastic Net      0.103 
Random Forest     0.74 
dtype: object

## The Utility Gain

We assume a mean-variance investor. He balance his portfolio between stock and risk-free bill monthly. The portfolio weights are decided by the prediction of the equity premium. The portfolio weights are given as

\begin{equation}
w_{j, t}=\left(\frac{1}{\gamma}\right)\left(\frac{\hat{r}_{t+1}}{\hat{\sigma}_{t+1}^2}\right)
\end{equation}

This weight is based on the prediction of the stock return and the variance of stock. We use ten years of rolling window for the estimation. This is in line with (Rapach 2010) and Campbell and Thompson (2008). The investor will gain an average utility over out-of-sample period as

\begin{equation}
\hat{v}_0=\hat{\mu}_0-\left(\frac{1}{2}\right) \gamma \hat{\sigma}_0^2
\end{equation}

To get the monthly portfolio weight we set $\gamma = 3$

In [7]:
def get_utility_gain_from_prediction(START_DATE: str,
                                     END_DATE: str,
                                     prediction: pd.DataFrame,
                                     historical_average: pd.DataFrame,
                                     equity_return: pd.DataFrame = None,
                                     rolling_window_size: int = 5, # number in year
                                     data_frequency: int = 12, # number of observations per year
                                     gamma: int = 3) -> float:
    """
    Get utility gain from prediction.
    TODO:
    -----
    1. the calculation of utility gain requires the true return of interested.
    2. we need to replace spy equity premium with the true equity return of interested.

    Parameters
    ----------
    START_DATE : str
        Start date of the utility gain curve.
        Format: YYYY-MM
    END_DATE : str
        End date of the utility gain curve.
        Format: YYYY-MM
    rolling_window_size : int, optional
        Rolling window size.
        Default: 10
    data_frequency : int, optional
        Data frequency.
        Default: 12
    gamma : int, optional
        Gamma.
        Default: 3

    Returns
    -------
    utility_gain : float
        Utility gain.
    """

    START_DATE = datetime.strptime(START_DATE, '%Y-%m')
    START_DATE = str(START_DATE.year - rolling_window_size) + '-' + str(START_DATE.month)
    data_frequency_dict = {12: 'monthly', 4: 'quarterly', 1: 'yearly'}
    econ_predictors = get_econ_predictors(START_DATE=START_DATE, END_DATE=END_DATE, data_freq=data_frequency_dict[data_frequency])
    risk_free_bond = econ_predictors['Treasury Bill'] / 100
    if equity_return is None:
        stock_return = econ_predictors['Equity Premium']
    else:
        stock_return = equity_return[START_DATE:END_DATE]
    portfolio_df = pd.concat([stock_return, risk_free_bond], axis=1).dropna()

    sample_varince = portfolio_df.iloc[:, 0].rolling(rolling_window_size * data_frequency - 1).var().dropna()
    varince_estimation = sample_varince.shift(1).dropna()

    stock_weight_0 = (1 / gamma) * (historical_average / varince_estimation)
    stock_weight_0 = stock_weight_0.clip(0, 1.5)
    portfolio_weight_0 = pd.concat([stock_weight_0, 1 - stock_weight_0], axis = 1)
    w_0 = portfolio_weight_0.values.reshape(-1, 1, 2)

    stock_weight_1 = (1 / gamma) * (prediction / varince_estimation)
    stock_weight_1 = stock_weight_1.clip(0, 1.5)
    portfolio_weight_1 = pd.concat([stock_weight_1, 1 - stock_weight_1], axis = 1)
    w_1 = portfolio_weight_1.values.reshape(-1, 1, 2)

    return_df = portfolio_df.loc[portfolio_weight_0.index] # need to change
    returns = return_df.values.reshape(-1, 2, 1)

    portfolio_return_0 = (w_0 @ returns).flatten()
    portfolio_return_1 = (w_1 @ returns).flatten()

    # annualize the return according to Rapach (2010)
    mu_0 = np.mean(portfolio_return_0) * data_frequency
    sigma_0 = np.var(portfolio_return_0) * data_frequency
    uitility_0 = mu_0 - 0.5 * gamma * sigma_0

    mu_1 = np.mean(portfolio_return_1) * data_frequency
    sigma_1 = np.var(portfolio_return_1) * data_frequency
    uitility_1 = mu_1 - 0.5 * gamma * sigma_1

    utility_gain = uitility_1 - uitility_0
    utility_gain_percentage = utility_gain * 100

    return utility_gain_percentage

In [27]:
utility_gain = prediction_df.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_df.index[0].strftime('%Y-%m'), 
                                                                                END_DATE=prediction_df.index[-1].strftime('%Y-%m'),
                                                                                historical_average=historical_average, 
                                                                                prediction=x))
utility_gain

Elastic Net      0.212202
Random Forest    0.047908
dtype: float64

In [28]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,R2,significance,Utility Gain
Elastic Net,0.078502,0.103,0.212202
Random Forest,-10.215775,0.74,0.047908


In [29]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='equity premium out-of-sample ml forecasting performance')

Save table to:../../table/


# Analysis of linear prediction

load prediction results from linear regression (combination)

In [4]:
prediction_df = pd.read_csv('../../data/prediction_linear_quarterly_1947_1965_2021.csv', index_col=0, parse_dates=True, date_parser=get_monthly_date_format)
START_DATE = '1965-01'
END_DATE = '2021-12'
prediction_df = prediction_df.loc[START_DATE:END_DATE]
equity_premium = prediction_df.pop('Equity Premium')
historical_average = prediction_df.pop('Historical Average')

1. R2

In [23]:
R_2_OOS = prediction_df.apply(lambda x: get_oos_r_square(y=equity_premium, y_hat=x, y_bar=historical_average))
R_2_OOS

Dividend Price Ratio    -0.244020
Dividend Yield          -0.365329
Earnings Price Ratio    -1.815205
Earnings Payout Ratio   -1.513486
Stock Variance          -6.280336
Book To Market          -0.777498
Net Equity Expansion    -3.776034
Treasury Bill            1.758113
Long Term Yield          2.555522
Long Term Return        -1.264156
Term Spread             -3.033965
Default Yield Spread    -0.971612
Default Return Spread   -4.607112
Inflation               -2.363594
Invest Capital Ratio     8.288966
Mean                     0.421112
Median                  -0.218530
Trimmed mean             0.356231
DMSPE theta 1            0.502269
DMSPE theta 0.9          1.112291
dtype: float64

2. significance of R2

In [24]:
significance = prediction_df.apply(lambda x: get_significance_of_MSPE_adjusted_test(y=equity_premium, y_hat=x, y_bar=historical_average))
significance

Dividend Price Ratio        0.631 
Dividend Yield              0.611 
Earnings Price Ratio        0.875 
Earnings Payout Ratio       0.811 
Stock Variance              0.136 
Book To Market              0.359 
Net Equity Expansion       0.059 *
Treasury Bill               0.209 
Long Term Yield            0.057 *
Long Term Return            0.903 
Term Spread                 0.768 
Default Yield Spread        0.217 
Default Return Spread       0.975 
Inflation                   0.649 
Invest Capital Ratio     0.003 ***
Mean                        0.509 
Median                      0.808 
Trimmed mean                0.545 
DMSPE theta 1               0.445 
DMSPE theta 0.9             0.137 
dtype: object

The utility gain

In [25]:
utility_gain = prediction_df.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_df.index[0].strftime('%Y-%m'), 
                                                                                END_DATE=prediction_df.index[-1].strftime('%Y-%m'),
                                                                                historical_average=historical_average, 
                                                                                prediction=x,
                                                                                data_frequency=4))
utility_gain

Dividend Price Ratio     2.216890
Dividend Yield           2.551894
Earnings Price Ratio     3.501774
Earnings Payout Ratio   -0.783380
Stock Variance          -2.155294
Book To Market           0.879638
Net Equity Expansion    -1.377711
Treasury Bill            1.386913
Long Term Yield          0.929047
Long Term Return         0.983267
Term Spread             -1.151260
Default Yield Spread    -0.786811
Default Return Spread    0.941280
Inflation               -0.286651
Invest Capital Ratio     4.764974
Mean                    -0.080955
Median                  -0.390945
Trimmed mean            -0.251698
DMSPE theta 1            0.046286
DMSPE theta 0.9          0.953536
dtype: float64

In [26]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,R2,significance,Utility Gain
Dividend Price Ratio,-0.244020,0.631,2.216890
Dividend Yield,-0.365329,0.611,2.551894
Earnings Price Ratio,-1.815205,0.875,3.501774
Earnings Payout Ratio,-1.513486,0.811,-0.783380
Stock Variance,-6.280336,0.136,-2.155294
Book To Market,-0.777498,0.359,0.879638
Net Equity Expansion,-3.776034,0.059 *,-1.377711
Treasury Bill,1.758113,0.209,1.386913
Long Term Yield,2.555522,0.057 *,0.929047
Long Term Return,-1.264156,0.903,0.983267


In [27]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='performance of linear prediction using quarterly data 2000 - 2021')

Save table to:../../table/


# Analysis of ML prediction

load ml prediction results

In [2]:
prediction_df = pd.read_csv('../../data/prediction_ml_quarterly_1947_1964_2021.csv', index_col=0, parse_dates=True, date_parser=get_monthly_date_format)
START_DATE = '1965-01'
END_DATE = '2021-12'
prediction_df = prediction_df.loc[START_DATE:END_DATE]
equity_premium = prediction_df.pop('Equity Premium')
historical_average = prediction_df.pop('Historical Average')

R2

In [28]:
R_2_OOS = prediction_df.apply(lambda x: get_oos_r_square(y=equity_premium, y_hat=x, y_bar=historical_average))
R_2_OOS

Elastic Net      0.723741
Random Forest   -3.972828
dtype: float64

significance of R2

In [29]:
significance = prediction_df.apply(lambda x: get_significance_of_MSPE_adjusted_test(y=equity_premium, y_hat=x, y_bar=historical_average))
significance

Elastic Net      0.003 ***
Random Forest     0.046 **
dtype: object

The utility gain

In [47]:
utility_gain = prediction_df.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_df.index[0].strftime('%Y-%m'), 
                                                                                END_DATE=prediction_df.index[-1].strftime('%Y-%m'),
                                                                                historical_average=historical_average, 
                                                                                prediction=x,
                                                                                data_frequency=4))
utility_gain

Elastic Net      0.407559
Random Forest    3.711325
dtype: float64

In [48]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,R2,significance,Utility Gain
Elastic Net,0.723741,0.297,0.407559
Random Forest,-3.972828,0.795,3.711325


In [49]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='performance of ml prediction using quarterly data 2000 - 2021')

Save table to:../../table/


# Analysis of implied expected return

load implied expected return

In [3]:
prediction_df = pd.read_csv('../../data/prediction_implied_quarterly_rl_1947_1964_2021.csv', index_col=0, parse_dates=True)
prediction_df.index = [get_monthly_date_format(x) for x in prediction_df.index]
prediction_df.columns = ['RL']

START_DATE = '1965-09'
END_DATE = '2021-12'
prediction_df = prediction_df[START_DATE:END_DATE]
equity_premium = equity_premium[START_DATE:END_DATE]
historical_average = historical_average[START_DATE:END_DATE]

R2

In [4]:
R_2_OOS = prediction_df.apply(lambda x: get_oos_r_square(y=equity_premium, y_hat=x, y_bar=historical_average))
R_2_OOS

RL    2.079368
dtype: float64

Significance of R2

In [5]:
get_p_value_of_MSPE_adjusted_test(y=equity_premium.values.flatten(), y_hat=prediction_df.values.flatten(), y_bar=historical_average.values.flatten())

0.02170097292246557

In [6]:
significance = prediction_df.apply(lambda x: get_significance_of_MSPE_adjusted_test(y=equity_premium, y_hat=x, y_bar=historical_average))
significance

RL    0.022 **
dtype: object

The utility gain

In [7]:
utility_gain = prediction_df.apply(lambda x: get_utility_gain_from_prediction(START_DATE=prediction_df.index[0].strftime('%Y-%m'), 
                                                                                END_DATE=prediction_df.index[-1].strftime('%Y-%m'),
                                                                                historical_average=historical_average, 
                                                                                prediction=x,
                                                                                data_frequency=4))
utility_gain

RL    2.346019
dtype: float64

Output results

In [8]:
evaluation_matric_df = pd.concat([R_2_OOS, significance, utility_gain], axis=1)
evaluation_matric_df.columns = ['R2', 'significance', 'Utility Gain']
evaluation_matric_df

,R2,significance,Utility Gain
RL,2.079368,0.022 **,2.346019


In [9]:
post_dataframe_to_latex_table(evaluation_matric_df, table_name='performance of implied prediction of RL using quarterly data 1965 - 2021')

Save table to:../../table/


# Replication of Rapach (2010) Analysis

In [49]:
R2_CL = pd.read_csv('../../data/linear_prediction_quarterly_R2_CL.csv', index_col=0, names=['CL'], header=0)
R2_CL_restricted = pd.read_csv('../../data/linear_prediction_quarterly_R2_CL_restricted.csv', index_col=0, names=['CL'], header=0)
R2_CL_restricted.columns = ['CL_restricted']
R2_MS = pd.read_csv('../../data/linear_prediction_quarterly_R2_MS.csv', index_col=0, names=['Rapach', 'MS'], header=0)
R2_MS.index = R2_CL.index.to_list()[:14] + ['I_K']

In [50]:
R2_total = pd.concat([R2_CL, R2_MS, R2_CL_restricted], axis=1)
R2_total.loc[['Mean', 'Median', 'Trimmed mean', 'DMSPE theta 1', 'DMSPE theta 0.9'], 'Rapach'] = [3.58, 3.04, 3.51, 3.54, 3.49]
R2_total

,CL,Rapach,MS,CL_restricted
Dividend Price Ratio,-0.380599,0.34,-0.722360,-0.380599
Dividend Yield,-0.443628,0.25,0.152974,-0.443628
Earnings Price Ratio,-0.367326,0.36,-2.332559,-0.367326
Earnings Payout Ratio,-1.472913,-1.42,-2.887096,-1.472913
Stock Variance,-10.576979,-12.97,-14.382389,-10.576979
Book To Market,-2.552643,-2.60,-5.113345,-2.552643
Net Equity Expansion,-0.900847,-0.91,-3.540437,-0.900847
Treasury Bill,-1.610106,-2.78,-0.695470,-1.610106
Long Term Yield,-2.415471,3.09,-0.289307,-2.415471
Long Term Return,0.602016,0.33,-1.113886,0.602016


In [43]:
R2_total.to_csv('../../data/linear_prediction_quarterly_R2_total.csv')